# **Fine Tuning and Ensembling Deep Architectures on Large Datasets in Resource Constrained Environment**

**The code in this file is used for fine tuning Resnet and Inception modules on ETHZ Food-101 dataset and test on UPMC Food-101 dataset. The base code is taken from the Deep Learning Programming Assignments and is modified and updated for the specific use of this project. Most of the rest of the code is our own work and the part where it is not is appropriately acknowledged.**

The authors of this file are Amama Mahmood, Heeyeon Kwon and Nafisa Ali Amir

**Section A: Prepare the environment**

```
Get the necessary installations, mount the drive and import packages
```




In [0]:
#@title
# Install pytorch and tqdm (if necessary)
!pip install torch
!pip install torchvision
!pip install tqdm

!pip install Pillow==4.0.0
!pip install PIL
!pip install image

In [0]:
# Mount your google drive as the data drive
# This will require google authorization
from google.colab import drive
drive.mount('/content/drive')

In [0]:
# Handle imports

import math
import os
import datetime
import csv
%matplotlib inline
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import datasets
from torchvision import transforms
from torch.autograd import Variable
import numpy as np
import tqdm
import torchvision.models as models

from IPython import display

import torch.utils.data as TUdata

import PIL
from PIL import Image
import os
import os.path
import numpy as np
from collections import defaultdict
import pandas as pd
import seaborn as sns

from sklearn.metrics import confusion_matrix
torch.set_default_tensor_type(torch.cuda.FloatTensor if torch.cuda.is_available() 
                                                     else torch.FloatTensor)

**Section B: Prepare Data for Food-101**


```
Upload images from drive.
Partition them into training and test (test here is actually the validation set)
Prepare the train and test loaders by giving the data path and setting arguments for doing normalisation and image pre-processing
```



In [0]:
#Uploading images from the drive
def default_loader(path):
	return Image.open(path).convert('RGB')

class FOOD101(TUdata.Dataset):
    def __init__(self, root, list_IDs, labels, transform=None):
        'Initialization'
        self.labels = labels
        self.list_IDs = list_IDs
        self.data_dir = os.path.join(root,'food101/images/')
        self.transform = transform

    def __len__(self):
        'Denotes the total number of samples'
        return len(self.list_IDs)

    def __getitem__(self, index):
        'Generates one sample of data'
        # Select sample
        ID = self.list_IDs[index]

        # Load data and get label ##example drive/My Drive/cs482/DL-project/food101/apple_pie/xxxxx.jpg
        img = default_loader(self.data_dir + ID[0] + '/' + ID[1] + '.jpg')
        if self.transform is not None:
            X = self.transform(img)
        y = self.labels[ID[0]]

        return X, y

In [0]:
def partition_data_and_index_labels(data_dir, dataset, classes_txt, train_txt, test_txt):
'''Partitioning data into train and test(validation actually) and indexing 
labels for all the categories'''
    labels_file = os.path.join(data_dir,dataset,classes_txt)
    labels = {}
    '''labels -> {apple_pie : 0, fish_and_chips : ...}'''
    with open(labels_file,'r') as rf:
        for idx, line in enumerate(rf.readlines()):
            line = line.strip()
            line = line
            labels[line] = idx
    
    train_file = os.path.join(data_dir,dataset,train_txt)
    test_file = os.path.join(data_dir,dataset,test_txt)
    partition = defaultdict(list)
    
    '''partition -> {'train : [[apple_pie, xxxxx.jpg], [apple_pie, xxyyyy.jpg], 
    [fish_and_chips, aaaabb.jpg]], test: [[apple_pie, abcdabcd.jpg]]}}'''
    
    with open(train_file,'r') as rf:
        for idx, line in enumerate(rf.readlines()):
            line = line.strip()
            ID = line.split('/')
            partition['train'].append([ID[0], ID[1]])
        

    with open(test_file,'r') as rf:
        for idx, line in enumerate(rf.readlines()):
            line = line.strip()
            ID = line.split('/')
            partition['test'].append([ID[0], ID[1]])
            
    return partition, labels

In [0]:
# Create the train and test loaders
def prepare_dataset(args):
    kwargs = {'num_workers': 1, 'pin_memory': True} if args.cuda else {}
    
    
    train_dataset = FOOD101(args.data_dir, args.partition['train'], args.labels, transform=transforms.Compose([transforms.RandomResizedCrop(args.resolution),
                                                                                                               transforms.RandomHorizontalFlip(),
                                                                                                               transforms.ToTensor(),
                                                                                                               transforms.Normalize(mean = [ 0.485, 0.456, 0.406 ],
                                                                                                                                  std = [ 0.229, 0.224, 0.225 ]) 
                                                                                                            ]))


    train_loader = TUdata.DataLoader(train_dataset, batch_size=args.batch_size, shuffle=True, **kwargs)


    test_dataset = FOOD101(args.data_dir, args.partition['test'], args.labels, transform=transforms.Compose([transforms.RandomResizedCrop(args.resolution),
                                                                                                             transforms.RandomHorizontalFlip(),
                                                                                                             transforms.ToTensor(),
                                                                                                             transforms.Normalize(mean = [ 0.485, 0.456, 0.406 ],
                                                                                                                                  std = [ 0.229, 0.224, 0.225 ]) 
                                                                                                            ]))

    test_loader = TUdata.DataLoader(test_dataset, batch_size=args.batch_size, shuffle=True, **kwargs)
    
    
    def time_stamp(fname, fmt='%m-%d-%H-%M_{fname}'):
        return datetime.datetime.now().strftime(fmt).format(fname=fname)
        
    training_run_name = time_stamp(args.dataset + '_' + args.name)
    training_run_dir = os.path.join(args.data_dir, 'run_dir', training_run_name)
    
    if not os.path.exists(training_run_dir):
        os.makedirs(training_run_dir)
    
    return train_loader, test_loader, train_dataset, test_dataset, training_run_dir

**Section C: Visualise Results in the form of Confusion Matrix**


```
The function cm_analysis is taken from ...
It allows us to visualise the results. This has been used for 10 category classification only.
```



In [0]:
#Function used for creating confusion matrix
#Acknowledgement to: https://gist.github.com/hitvoice/36cf44689065ca9b927431546381a3f7
#Updated to meet requirements 
def cm_analysis(y_true, y_pred, figsize=(10,10)):
    """
    Generate matrix plot of confusion matrix with pretty annotations.
    The plot image is saved to disk.
    args: 
      y_true:    true label of the data, with shape (nsamples,)
      y_pred:    prediction of the data, with shape (nsamples,)
      filename:  filename of figure file to save
      labels:    string array, name the order of class labels in the confusion matrix.
                 use `clf.classes_` if using scikit-learn models.
                 with shape (nclass,).
      ymap:      dict: any -> string, length == nclass.
                 if not None, map the labels & ys to more understandable strings.
                 Caution: original y_true, y_pred and labels must align.
      figsize:   the size of the figure plotted.
    """
    labels=map(str, range(9 + 1))
    labels_file = os.path.join(args.data_dir,args.dataset,'meta/C10.txt')
    ymap = {}
    '''labels -> {apple_pie : 0, fish_and_chips : 4}'''
    with open(labels_file,'r') as rf:
        for idx, line in enumerate(rf.readlines()):
            line = line.strip()
            line = line
            ymap[idx] = line
    if ymap is not None:
        y_pred = [ymap[np.int(yi)] for yi in y_pred]
        y_true = [ymap[np.int(yi)] for yi in y_true]
        labels = [ymap[np.int(yi)] for yi in labels]
    cm = confusion_matrix(y_true, y_pred, labels=labels)
    cm_sum = np.sum(cm, axis=1, keepdims=True)
    cm_perc = cm / cm_sum.astype(float) * 100
    annot = np.empty_like(cm).astype(str)
    nrows, ncols = cm.shape
    for i in range(nrows):
        for j in range(ncols):
            c = cm[i, j]
            p = cm_perc[i, j]
            if i == j:
                s = cm_sum[i]
                annot[i, j] = '%.1f%%\n%d/%d' % (p, c, s)
            elif c == 0:
                annot[i, j] = ''
            else:
                annot[i, j] = '%.1f%%\n%d' % (p, c)
    cm = pd.DataFrame(cm, index=labels, columns=labels)
    cm.index.name = 'Actual'
    cm.columns.name = 'Predicted'
    fig, ax = plt.subplots(figsize=figsize)
    sns.heatmap(cm, annot=annot, fmt='', ax=ax)
    plt.show
    #plt.savefig(filename)

**Section D: Parameters**


```
The class Args handles all parameters. 
```



In [0]:
# The Args object will contain all of our parameters
# If you want to run with different arguments, create another Args object

class Args(object):
    def __init__(self, name='food101', batch_size=64, test_batch_size=1000,
            epochs=50, lr=0.0001, optimizer='adam', momentum=0.5,
            seed=1, log_interval=100, dataset='food101', n_categories=101,
            data_dir='drive/My Drive/DL-project/', model='ResNet152',
            cuda=True):
        self.name = name # name for this training run. Don't use spaces.
        self.batch_size = batch_size
        self.test_batch_size = test_batch_size # Input batch size for testing
        self.epochs = epochs # Number of epochs to train
        self.lr = lr # Learning rate
        self.optimizer = optimizer # sgd/p1sgd/adam/rms_prop
        self.momentum = momentum # SGD Momentum
        self.seed = seed # Random seed
        self.log_interval = log_interval # Batches to wait before logging
                                     # detailed status. 0 = never
        self.dataset = dataset # mnist/fashion_mnist
        self.data_dir = data_dir
        self.model = model 
        self.resolution = 299 if self.model == 'inception' else 224
        self.cuda = cuda and torch.cuda.is_available()
        self.categories = n_categories
        if self.categories == 10:
            self.classes_txt = 'meta/C10.txt'
            self.train_txt = 'meta/train_C10.txt'
            self.test_txt = 'meta/test_C10.txt'
        else:
            self.classes_txt = 'meta/classes.txt'
            self.train_txt = 'meta/train.txt'
            self.test_txt = 'meta/test.txt'
        
        self.partition, self.labels = partition_data_and_index_labels(self.data_dir,self.dataset,self.classes_txt,self.train_txt,self.test_txt)

In [0]:
#To print the number of GPU's available
if torch.cuda.is_available():
    print(torch.cuda.device_count(),"GPUs available!!!")
else:
    print("GPU not available!!!")

**Section E: Training, Validation and Accuracy Computation**
```  
Models supported are Inception-v3, ResNet-18, ResNet-34, ResNet-50, ResNet-101, ResNet-152

Optimizers supported are Adam, SGD and RMSProp

The function run_experiment calls train and test functions for training and validation respectively for every epoch. After each epoch, checkpoints are created by saving the model parameters.

The argument resume is added to run_experiment, which allows training in parts and resuming from the last saved checkpoint when set to 1.

Once max epochs is reached, the final checkpoint is saved, top-1 and top-5 accuracies are computed and training and validation accuracy plots are generated and saved.
```




In [0]:
def train(model, optimizer, train_loader, epoch, total_minibatch_count,
        train_losses, train_accs,args):
    # Training for a full epoch

    model.train()
    correct_count, total_loss, total_acc = 0., 0., 0.
    progress_bar = tqdm.tqdm(train_loader, desc='Training')
    CEloss = nn.CrossEntropyLoss()
    for batch_idx, (data, target) in enumerate(progress_bar):

        if args.cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)

        optimizer.zero_grad()

        # Forward prediction step
        #Inception model needs special handling: refer https://github.com/pytorch/vision/issues/302
        if args.model == 'inception':
            output,_ = model(data)
        else:
            output= model(data) 
        loss = CEloss(output, target)

        # Backpropagation step
        loss.backward()
        optimizer.step()

        # The batch has ended, determine the accuracy of the predicted outputs
        pred = output.data.max(1)[1]  

        # target labels and predictions are categorical values from 0 to 9.
        matches = target == pred
        accuracy = matches.float().mean()
        correct_count += matches.sum()

        if args.log_interval != 0 and \
                total_minibatch_count % args.log_interval == 0:

            train_losses.append(loss.data[0])
            train_accs.append(accuracy.data[0])
            
        total_loss += loss.data
        total_acc += accuracy.data
            
        progress_bar.set_description(
            'Epoch: {} loss: {:.4f}, acc: {:.2f}'.format(
                epoch, total_loss / (batch_idx + 1), total_acc / (batch_idx + 1)))
        #progress_bar.refresh()

        total_minibatch_count += 1

    return total_minibatch_count

In [0]:
#Computes Top 1 and Top k accuracy
#Acknowlegment to: https://github.com/pytorch/examples/blob/master/imagenet/main.py
def accuracy(output, target, topk=(1,)):
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res

In [0]:
def test(model, test_loader, epoch, total_minibatch_count,
        val_losses, val_accs,args):
    # Validation Testing
    model.eval()
    test_loss, correct = 0., 0.
    progress_bar = tqdm.tqdm(test_loader, desc='Validation')
    CEloss = nn.CrossEntropyLoss()
    #if epoch==args.epochs:
    if args.cuda:
      cum_pred=torch.cuda.LongTensor([])
      cum_target=torch.cuda.LongTensor([])
      cum_output=torch.cuda.FloatTensor([])
    else:
      cum_pred=torch.LongTensor([])
      cum_target=torch.LongTensor([])
      cum_output=torch.FloatTensor([])
    with torch.no_grad():
        for data, target in progress_bar:
            if args.cuda:
                data, target = data.cuda(), target.cuda()
            data, target = Variable(data), Variable(target)
            #Inception model needs special handling: refer https://github.com/pytorch/vision/issues/302
            if args.model == 'inception':
                output = model(data)
            else:
                output = model(data) 
            test_loss += CEloss(output, target).data  # sum up batch loss
            pred = output.data.max(1)[1]  # get the index of the max log-probability
            if epoch==args.epochs:
              cum_output=torch.cat((cum_output, output))
              cum_pred= torch.cat((cum_pred, pred))
              cum_target=torch.cat((cum_target, target))
            correct += (target == pred).float().sum()
    
#    print('\nConfusion_matrix:\n', confusion)
    if epoch==args.epochs:
      #cm_analysis(cum_target, cum_pred)
      acc1, acc5 = accuracy(cum_output, cum_target, topk=(1, 5))
      print ('\n\nTop 5 accuracy is  : ', acc5,' while Top 1 accuracy is :', acc1,'\n')
      
      ## only uncomment when target file needs to be saved for upmc or food101 testset, edit accordingly
      #path2='/content/drive/My Drive/DL-project/probabilities/'
      #cum_target=cum_target.cpu().numpy()
      #df = pd.DataFrame(cum_target)
      #df.to_csv(os.path.join(path2+'upmc_'+args.model+'_target.csv'))
      #print('\npred: ',cum_pred, '\ntarget: ', cum_target)
    
    
    test_loss /= len(test_loader.dataset)
    
    acc = correct / len(test_loader.dataset)

    
    
    val_losses.append(test_loss)
    val_accs.append(acc)
    
    progress_bar.clear()
    progress_bar.write(
        '\nEpoch: {} validation test results - Average val_loss: {:.4f}, val_acc: {}/{} ({:.2f}%)'.format(
            epoch, test_loss, correct, len(test_loader.dataset),
            100. * correct / len(test_loader.dataset)))
    return acc,cum_output

In [0]:
# Run the experiment
def run_experiment(args,resume=0):

    total_minibatch_count = 0

    torch.manual_seed(args.seed)
    if args.cuda:
        torch.cuda.manual_seed(args.seed)

    train_loader, test_loader, _, _, run_path = prepare_dataset(args)

    
    epochs_to_run = args.epochs
    
    

    # Choose model
    if args.model == 'inception':
        model = models.inception_v3(pretrained=True)
    elif args.model == 'ResNet18':
        model = models.resnet18(pretrained=True)
    elif args.model=='ResNet34':
        model=models.resnet34(pretrained=True)
    elif args.model=='ResNet50':
        model=models.resnet50(pretrained=True)
    elif args.model=='ResNet101':
        model=models.resnet101(pretrained=True)
    elif args.model=='ResNet152':
        model=models.resnet152(pretrained=True)
    elif args.model in globals():
        model = globals()[args.model]()
    else:
        raise ValueError('Unknown model type: ' + args.model)
    ##updating fc layer to make 101 classes rather than 1000
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, 101)
    if args.cuda:
        model.cuda()

    # Choose optimizer
    if args.optimizer == 'sgd':
        optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=args.momentum)
    elif args.optimizer == 'adam':
        optimizer = optim.Adam(model.parameters(), lr=args.lr)
    elif args.optimizer == 'rmsprop':
        optimizer = optim.RMSprop(model.parameters())
    else:
        raise ValueError('Unsupported optimizer: ' + args.optimizer)

    # Run the primary training loop, starting with validation accuracy of 0
    val_acc = 0
    train_losses, train_accs = [], []
    val_losses, val_accs = [], []
    last_epoch=1
    if resume==1:
      path='/content/drive/My Drive/DL-project/checkpoints/' ## change the path to where thw checkpoint file is
      path=os.path.join(path + args.model+'_lastest.pkl') 
      cuda = torch.cuda.is_available()
      if cuda:
        checkpoint = torch.load(path)
      else:
          # Load GPU model on CPU
          checkpoint = torch.load(path,
                                  map_location=lambda storage,
                                  loc: storage)
      model.load_state_dict(checkpoint['state_dict'])
      last_epoch = checkpoint['epoch'] +1
    

    for epoch in range(last_epoch, epochs_to_run + 1):
        
        # train for 1 epoch
        total_minibatch_count = train(model, optimizer, train_loader,
                                    epoch, total_minibatch_count,
                                    train_losses, train_accs,args)
        # validate progress on test dataset
        val_acc,_= test(model, test_loader, epoch, total_minibatch_count,
                       val_losses, val_accs)
        path='/content/drive/My Drive/DL-project/checkpoints/'       ##change the path where you wanna save the checkpoint file
        state={
        'epoch': epoch,
        'state_dict': model.state_dict(),
        'best_accuracy': val_acc,
        'args': args
        
    }
        torch.save(state, os.path.join(path + args.model+'_epoch'+str(epoch)+'.pkl'))
        torch.save(state, os.path.join(path + args.model+'_lastest1.pkl'))
    fig, axes = plt.subplots(1,4, figsize=(13,4))
    # plot the losses and acc
    plt.title(args.name)
    axes[0].plot(train_losses)
    axes[0].set_title("Loss")
    axes[1].plot(train_accs)
    axes[1].set_title("Acc")
    axes[2].plot(val_losses)
    axes[2].set_title("Val loss")
    axes[3].plot(val_accs)
    axes[3].set_title("Val Acc")
    
    # Write to csv file
    with open(os.path.join(run_path + 'train.csv'), 'w') as f:
        csvw = csv.writer(f, delimiter=',')
        for loss, acc in zip(train_losses, train_accs):
            csvw.writerow((loss, acc))

    # Predict and Test
    images, labels = next(iter(test_loader))
    if args.cuda:
        images, labels = images.cuda(), labels.cuda()
    output = model(images)
    predicted = torch.max(output, 1)[1]
    fig, axes = plt.subplots(1,6)
    for i, (axis, img, lbl) in enumerate(zip(axes, images, predicted)):
        if i > 5:
            break
        img = img.permute(1,2,0).squeeze()
        axis.imshow(img)
        axis.set_title(lbl.data)
        axis.set_yticklabels([])
        axis.set_xticklabels([])
            
    
        
        

**The command to start the training**






In [0]:
run_experiment(Args(model='ResNet152',n_categories=101,epochs=10,batch_size=8,lr=0.0001,optimizer='adam'),resume=0)

**Section F: Ensembling Resnet-152 and Inception v3**
```  
This part of code used to compute the validation accuracy on last epoch for unshuffled test images and saves the output into .csv files along with target labels which are needed for ensembling later on.





In [0]:
model='inception'
path='/content/drive/My Drive/DL-project/checkpoints/' ## change the path to where thw checkpoint file is
path2='/content/drive/My Drive/DL-project/probabilities/'
path=os.path.join(path + model+'_latest.pkl')                       ## change the filename if needed
cuda = torch.cuda.is_available()
if cuda:
    checkpoint = torch.load(path)
else:
    # Load GPU model on CPU
    checkpoint = torch.load(path,
                            map_location=lambda storage,
                            loc: storage)
args= checkpoint['args']
args.data_dir='drive/My Drive/DL-project/'
args.epochs=checkpoint['epoch']
kwargs = {'num_workers': 1, 'pin_memory': True} if args.cuda else {}
test_dataset = FOOD101(args.data_dir, args.partition['test'], args.labels, transform=transforms.Compose([transforms.RandomResizedCrop(args.resolution),
                                                                                                         transforms.RandomHorizontalFlip(),
                                                                                                         transforms.ToTensor(),
                                                                                                         transforms.Normalize(mean = [ 0.485, 0.456, 0.406 ],
                                                                                                                            std = [ 0.229, 0.224, 0.225 ]) 
                                                                                                        ]))
test_loader = TUdata.DataLoader(test_dataset, batch_size=args.batch_size, shuffle=False, **kwargs)
# Choose model
if args.model == 'default' or args.model == 'P2Q7DefaultChannelsNet':
    model = Net()
elif args.model == 'inception':
    model = models.inception_v3(pretrained=True)
elif args.model == 'ResNet18':
    model = models.resnet18(pretrained=True)
elif args.model=='ResNet34':
    model=models.resnet34(pretrained=True)
elif args.model=='ResNet50':
    model=models.resnet50(pretrained=True)
elif args.model=='ResNet101':
    model=models.resnet101(pretrained=True)
elif args.model=='ResNet152':
    model=models.resnet152(pretrained=True)
elif args.model in globals():
    model = globals()[args.model]()
else:
    raise ValueError('Unknown model type: ' + args.model)

num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 101)
#print(model)
if args.cuda:
    model.cuda()
    model = torch.nn.DataParallel(model).cuda()  #comment it out while loading Resnet-152 or if key mismatch error occurs in state_dict
epoch = checkpoint['epoch']
model.load_state_dict(checkpoint['state_dict'])
total_minibatch_count= 0
print("=> loaded checkpoint '{}' (trained for {} epochs)".format(path, checkpoint['epoch']),'\n')


val_losses, val_accs = [], []
val_acc,output = test(model, test_loader, epoch, total_minibatch_count,
                   val_losses, val_accs,args)

path2='/content/drive/My Drive/DL-project/probabilities/'
output=output.cpu().numpy()
df = pd.DataFrame(output)
df.to_csv(os.path.join(path2+'inception_latest.csv'))     #change the file to be saved to according to the model


In [0]:
path2='/content/drive/My Drive/DL-project/probabilities/'
print('loading inception')
prob_inc = pd.read_csv(os.path.join(path2+'inception_latest.csv'), sep=",");
prob_inc=prob_inc.values[:,1:]
prob_inc=torch.tensor(prob_inc)
prob_inc=(prob_inc-prob_inc.min(1,keepdim=True)[0]) / (prob_inc.max(1, keepdim=True)[0]-prob_inc.min(1,keepdim=True)[0])

print('loading residual')
prob_res = pd.read_csv(os.path.join(path2+'resnet_latest_10epochs.csv'), sep=",");
prob_res=prob_res.values[:,1:]
prob_res=torch.tensor(prob_res)
prob_res=(prob_res-prob_res.min(1,keepdim=True)[0]) / (prob_res.max(1, keepdim=True)[0]-prob_res.min(1,keepdim=True)[0])

print('loading_target')
target = pd.read_csv(os.path.join(path2+'_inception_target.csv'), sep=",");
target=torch.cuda.LongTensor(target.values[:,1:])
#Ensemble
final_prob=prob_inc*0.5+prob_res*0.5
final_prob=torch.tensor(final_prob)

acc1, acc5 = accuracy(final_prob, target, topk=(1, 5))

print(acc1,acc5)



In [0]:
#Uploading images from drive
class UPMC_FOOD101(TUdata.Dataset):
    def __init__(self, root, list_IDs, labels, transform=None):
        'Initialization'
        self.labels = labels
        self.list_IDs = list_IDs
        self.data_dir = os.path.join(root,'upmc_food101/images/')
        self.transform = transform

    def __len__(self):
        'Denotes the total number of samples'
        return len(self.list_IDs)

    def __getitem__(self, index):
        'Generates one sample of data'
        # Select sample
        ID = self.list_IDs[index]

        # Load data and get label ##example drive/My Drive/cs482/DL-project/upmc_food101/train/apple_pie/xxxxx.jpg
        img = default_loader(self.data_dir + ID[0] + '/' + ID[1] + '/' + ID[2])
        if self.transform is not None:
            X = self.transform(img)
        y = self.labels[ID[1]]

        return X, y


**Section G: Evaluation on UPMC Food-101**
```  
Prepare dataset by uploading images and creating the test loader in a similar manner as was done for Food-101

The same models are supported here.

There is no training or validation here. Only one round of test which gives a prediction of the class associated with the image.

It eventually prints the top-1 and top-5 accuracy.
```





In [0]:
def evaluate_upmc_food101(args):
    torch.manual_seed(args.seed)
    if args.cuda:
        torch.cuda.manual_seed(args.seed)

    #Give the path of the image list file (all images from upmc)
    upmc_test_file = os.path.join(args.data_dir,'upmc_food101/images/upmc_testset.txt')
    partition = defaultdict(list)
    with open(upmc_test_file,'r') as rf:
        for idx, line in enumerate(rf.readlines()):
            line = line.strip()
            ID = line.split('/')
            partition['upmc_test'].append([ID[0], ID[1], ID[2]])

    upmc_test_dataset = UPMC_FOOD101(args.data_dir, partition['upmc_test'], args.labels, transform=transforms.Compose([transforms.RandomResizedCrop(args.resolution),
                                                                                                               transforms.RandomHorizontalFlip(),
                                                                                                               transforms.ToTensor(),
                                                                                                               transforms.Normalize(mean = [ 0.485, 0.456, 0.406 ],
                                                                                                                                  std = [ 0.229, 0.224, 0.225 ]) 
                                                                                                            ]))

    kwargs = {'num_workers': 1, 'pin_memory': True} if args.cuda else {}
    upmc_test_loader = TUdata.DataLoader(upmc_test_dataset, batch_size=args.batch_size, shuffle=False, **kwargs)

    # Choose model
    if args.model == 'inception':
        model = models.inception_v3(pretrained=True)
    elif args.model == 'ResNet18':
        model = models.resnet18(pretrained=True)
    elif args.model=='ResNet34':
        model=models.resnet34(pretrained=True)
    elif args.model=='ResNet50':
        model=models.resnet50(pretrained=True)
    elif args.model=='ResNet101':
        model=models.resnet101(pretrained=True)
    elif args.model=='ResNet152':
        model=models.resnet152(pretrained=True)
    elif args.model in globals():
        model = globals()[args.model]()
    else:
        raise ValueError('Unknown model type: ' + args.model)
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, 101)
    if args.cuda:
        model.cuda()
        model = torch.nn.DataParallel(model).cuda()         #comment out in case of resnet or if error is in loading state_dict
    path = os.path.join(args.data_dir, 'checkpoints', args.model + '_latest.pkl')
    cuda = torch.cuda.is_available()
    if cuda:
        checkpoint = torch.load(path)
    else:
        # Load GPU model on CPU
        checkpoint = torch.load(path,
                            map_location=lambda storage,
                            loc: storage)
    model.load_state_dict(checkpoint['state_dict'])
    print("=> loaded checkpoint '{}' (trained for {} epochs)".format(path, checkpoint['epoch']),'\n')


    # validate progress on test dataset
    test_losses, test_accs = [],[]
    test_acc,output = test(model, upmc_test_loader, args.epochs, 0,
                       test_losses, test_accs,args)
    path2='/content/drive/My Drive/DL-project/probabilities/'
    output=output.cpu().numpy()
    df = pd.DataFrame(output)
    df.to_csv(os.path.join(path2+'upmc_inception_latest.csv')) # change as needed
    

**Command to run the evaluation code**






In [0]:
evaluate_upmc_food101(Args(model='inception',n_categories=101,epochs=30,batch_size=64,lr=0.0001))

** Section H: Ensembling test result on UPMC-food101 dataset**

Loading csv files and then ensembling the output of two models to give results on testset of UPMC-food101.


In [0]:

path2='/content/drive/My Drive/DL-project/probabilities/'

print('loading inception')
prob_inc = pd.read_csv(os.path.join(path2+'upmc_inception_latest.csv'), sep=",");
prob_inc=prob_inc.values[:,1:]
prob_inc=torch.tensor(prob_inc)
prob_inc=(prob_inc-prob_inc.min(1,keepdim=True)[0]) / (prob_inc.max(1, keepdim=True)[0]-prob_inc.min(1,keepdim=True)[0])

print('loading residual')
prob_res = pd.read_csv(os.path.join(path2+'umpc_resnet_latest.csv'), sep=",");
prob_res=prob_res.values[:,1:]
prob_res=torch.tensor(prob_res)
prob_res=(prob_res-prob_res.min(1,keepdim=True)[0]) / (prob_res.max(1, keepdim=True)[0]-prob_res.min(1,keepdim=True)[0])

print('loading_target')
target = pd.read_csv(os.path.join(path2+'upmc_inception_target.csv'), sep=",");
target=torch.cuda.LongTensor(target.values[:,1:])
#Ensemble
final_prob=prob_inc*0.5+prob_res*0.5
final_prob=torch.tensor(final_prob)

acc1, acc5 = accuracy(final_prob, target, topk=(1, 5))

print(acc1,acc5)

